In [1]:
from tqdm import tqdm

In [2]:
with open('input.txt')as fl:
    lns = fl.readlines()
puzzles = []
part2 = True # change for part 1 vs 2
for ln in lns:
    springs, counts = ln.strip().split(' ')
    if part2:
        springs = '?'.join([springs]*5)
        counts = ','.join([counts]*5)
    counts = tuple(int(i) for i in counts.split(','))
    puzzles.append((springs, counts))

In [3]:
# tool to get the next contig from a start but we can't skip a # and if last dont
# leave any left and if first dont skip the first #
def get_next_contig(start, count, springs, dontskip = False, last = False, first = False):
    for i in range(start, len(springs) - count + 1):
        if dontskip and i - 1 > 0 and springs[i-1] == '#':
            return -1
        if all( v in {'#','?'} for v in springs[i:(i+count)]) and \
                not ( (i - 1 >= 0 and springs[i-1] == '#') or \
                   ( (i+count)<len(springs) and springs[i+count] == '#') ) and \
                not (last and any( v == '#' for v in springs[(i+count):]) ) and \
                not (first and any(v == '#' for v in springs[:i])):
            return i
    return -1

In [4]:
# recursively count the contigs past current point using caching for speed 
def count_contigs(start, counts, springs, n=None, cache = None):
    total, i, c = 0, 0, counts[0]
    n = len(counts) if n is None else n
    cache = dict() if cache is None else cache
    while True:
        i = get_next_contig(start, c, springs, dontskip=True, 
                            last=len(counts)==1, first=len(counts)==n)
        if i == -1:
            break # bail out because this is impossible
        else:
            if len(counts) > 1:
                cnt = cache.get((i+c+1, tuple(counts[1:])), None)
                if cnt is None:
                    cnt = count_contigs(i+c+1, counts[1:], springs, n=n, cache=cache)
                    cache[(i+c+1, tuple(counts[1:]))] = cnt
                total += cnt
            else:
                total += 1
        start = i+1
    return total

In [5]:
sum(count_contigs(0, counts, springs) for springs, counts in tqdm(puzzles))

100%|██████████| 1000/1000 [00:01<00:00, 859.01it/s]


6555315065024